# Meshed AC-DC example

This example has a 3-node AC network coupled via AC-DC converters to a 3-node DC network. There is also a single point-to-point DC using the Link component.

The data files for this example are in the examples folder of the github repository: <https://github.com/PyPSA/PyPSA>.

In [ ]:
import matplotlib.pyplot as plt

import pypsa

%matplotlib inline
plt.rc("figure", figsize=(8, 8))

In [ ]:
network = pypsa.examples.ac_dc_meshed(from_master=True)

In [ ]:
# get current type (AC or DC) of the lines from the buses
lines_current_type = network.lines.bus0.map(network.buses.carrier)
lines_current_type

In [ ]:
network.plot(
    line_colors=lines_current_type.map(lambda ct: "r" if ct == "DC" else "b"),
    title="Mixed AC (blue) - DC (red) network - DC (cyan)",
    color_geomap=True,
    jitter=0.3,
)
plt.tight_layout()

In [ ]:
network.links.loc["Norwich Converter", "p_nom_extendable"] = False

We inspect the topology of the network. Therefore use the function `determine_network_topology` and inspect the subnetworks in `network.sub_networks`.

In [ ]:
network.determine_network_topology()
network.sub_networks["n_branches"] = [
    len(sn.branches()) for sn in network.sub_networks.obj
]
network.sub_networks["n_buses"] = [len(sn.buses()) for sn in network.sub_networks.obj]

network.sub_networks

The network covers 10 time steps. These are given by the `snapshots` attribute.

In [ ]:
network.snapshots

There are 6 generators in the network, 3 wind and 3 gas. All are attached to buses:

In [ ]:
network.generators

We see that the generators have different capital and marginal costs. All of them have a `p_nom_extendable` set to `True`, meaning that capacities can be extended in the optimization.

The wind generators have a per unit limit for each time step, given by the weather potentials at the site. 

In [ ]:
network.generators_t.p_max_pu.plot.area(subplots=True)
plt.tight_layout()

Alright now we know how the network looks like, where the generators and lines are. Now, let's perform a optimization of the operation and capacities.

In [ ]:
network.optimize();

The objective is given by:

In [ ]:
network.objective

Why is this number negative? It considers the starting point of the optimization, thus the existent capacities given by `network.generators.p_nom` are taken into account.  

The real system cost are given by

In [ ]:
network.objective + network.objective_constant

The optimal capacities are given by `p_nom_opt` for generators, links and storages and `s_nom_opt` for lines.

Let's look how the optimal capacities for the generators look like.

In [ ]:
network.generators.p_nom_opt.div(1e3).plot.bar(ylabel="GW", figsize=(8, 3))
plt.tight_layout()

Their production is again given as a time-series in `network.generators_t`.

In [ ]:
network.generators_t.p.div(1e3).plot.area(subplots=True, ylabel="GW")
plt.tight_layout()

What are the Locational Marginal Prices in the network. From the optimization these are given for each bus and snapshot.

In [ ]:
network.buses_t.marginal_price.mean(1).plot.area(figsize=(8, 3), ylabel="Euro per MWh")
plt.tight_layout()

We can inspect further quantities as the active power of AC-DC converters and HVDC link.

In [ ]:
network.links_t.p0

In [ ]:
network.lines_t.p0

...or the active power injection per bus.

In [ ]:
network.buses_t.p